# ブリックを実装する

これは，[このコード](https://github.com/nssuperx/irl334-research-srcs/blob/main/multivalue/play.py)とそれに付随するモジュールのipynb版です．

PyTorchを使用します．

## 1つのブリックを作る

何次元かの入力を受け取って，興奮したかしてないかを出力する素子を作ります．
* 入力するデータを用意
* ブリックの入力素子に入力
* 状態を選択
* 出力

の順で説明します．

### 入力するデータの用意

PyTorch DataLoaderを使用します．

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

データセットはMNISTを使います．

In [ ]:
datasets_path: str = "../pt_datasets"

train_dataset = datasets.MNIST(
    root=datasets_path,
    train=True,
    download=True,
    transform=ToTensor(),
)
test_dataset = datasets.MNIST(
    root=datasets_path,
    train=False,
    download=True,
    transform=ToTensor(),
)

MNIST_classes = datasets.MNIST.classes

In [ ]:
print(train_dataset)
print(MNIST_classes)

MNISTではなく，もっと小さいデータを扱いたいなら，以下のものを使用してください．縦棒のデータセットを作ります．
小さいデータセットなので，学習，テストで分けていません．
モデルの動作確認のみに使用してください．**ベンチマークで使用しない**でください．インチキです．

In [ ]:
from typing import Tuple
import itertools
import torch
from torch.utils.data import Dataset

class VerticalLine(Dataset):
    def __init__(self) -> None:
        super().__init__()
        pattern = (
            (0, 0, 0),
            (1, 0, 0),
            (0, 1, 0),
            (0, 0, 1))
        choices = itertools.product(range(4), repeat=3)
        correct_pattern_choice = ((1, 1, 1), (2, 2, 2), (3, 3, 3))
        self.data: list[tuple[torch.Tensor, int]] = []

        for c in choices:
            array_tmp = []
            for i in c:
                array_tmp.append(pattern[i])
            self.data.append((torch.Tensor(array_tmp), 1 if c in correct_pattern_choice else 0))

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, int]:
        return self.data[index][0], self.data[index][1]

    def __len__(self) -> int:
        return len(self.data)

In [ ]:
train_datasets = VerticalLine()
test_datasets = VerticalLine()

データセットを読み込みました．次はこれを扱いやすくします．PyTorchではDataLoaderを使います．

In [ ]:
batch_size: int = 10

trainloader = DataLoader(train_dataset, batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size, shuffle=False)

In [1]:
print(trainloader)

興味があるなら中身を見てみましょう．

In [ ]:
tl_iter = iter(trainloader)
data = next(tl_iter)
print(data)
print(type(data))
print(len(data))

上記の手順は，簡単に説明すると，Pythonのfor文の1回目の動作を手作業でやっています．
普通はやるべきではないです．

DataLoaderは要素数2のリストを返してくるようです．

In [ ]:
print(type(data[0]))
print(type(data[1]))
print(f"data[0] size: {len(data[0])}")
print(f"data[1] size: {len(data[1])}")
print(data[0][0])
print(data[1][0])

もっと中身を見てみます．
中身は入力として与えるTensorと，正解ラベルのTensorの組です．
要素数はそれぞれDataLoaderを作るときに与えたバッチサイズの値です．

まとめると，DataLoaderは以下のことをします．
* 入力と正解の組を返す
* それぞれの型はTensor，要素数はバッチサイズ

詳しくは[Pytorchのリファレンス](https://pytorch.org/docs/stable/data.html)を見てください．
[Tensor](https://pytorch.org/docs/stable/tensors.html)とは，PyTorchで扱う様々な機能がある便利な配列型です．

これでデータを扱う準備ができました．

実験をする前に，DataLoaderをもとの状態に戻します．

In [ ]:
batch_size: int = 10

trainloader = DataLoader(train_dataset, batch_size, shuffle=True)
testloader = DataLoader(test_dataset, batch_size, shuffle=False)

### ブリックに入力する

ブリックへの入力を説明しますが，その前に少しだけPyTorchの使い方を説明します．

#### `nn.module`

最近の深層神経回路モデルでは，ニューロンの**層**を何個も重ねて回路を構築します．
PyTorchでは回路を**モジュール**を組み合わせて構築します．
普段の実験では，この**モジュール**に入力を与えて出力させます．
手っ取り早く使うための知識と方法を説明します．

* モジュールは **`nn.Module`を継承したクラス**
* 何らかのパラメータ（重み weight）などを持たせられる
* **`forward`関数が実装されてないといけない**（[インターフェース](https://ja.wikipedia.org/wiki/%E3%82%A4%E3%83%B3%E3%82%BF%E3%83%95%E3%82%A7%E3%83%BC%E3%82%B9_(%E6%8A%BD%E8%B1%A1%E5%9E%8B))を知っているとよい．この考え方に近い．）
* `forward`関数は入力としてTensor型のベクトルを受け取り，Tensor型のベクトルを返す必要がある


詳しいことは[リファレンス](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)を見てください．